In [1]:
import gym
from gym_power_trading.envs import PowerTradingEnv
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

In [5]:
file_name = 'data\AEP_PSGC1_AMP_long.h5'
path = Path(file_name).resolve()
uri = str(path)

In [3]:
df = pd.read_hdf(uri)
df.head()

HE
MARKET_DAY NODE          TYPE    VALUE HourEnding       
2021-07-01 AEP.PSGC1.AMP Gennode LMP   1           25.88
                                       2           24.88
                                       3           23.45
                                       4           22.47
                                       5           21.73

In [2]:
my_dataframe = pd.read_csv("RandomCloseValues.csv")

In [3]:
env = PowerTradingEnv(df=my_dataframe, window_size=3, frame_bound=(10, 100))
obs = env.reset()
for _ in tqdm(range(100)):
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)
    print(info)
    if truncated:
        obs = env.reset()

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 2 and the array at index 1 has size 3

In [4]:
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env

In [5]:
venv = make_vec_env(lambda: PowerTradingEnv(df=my_dataframe, window_size=3, frame_bound=(10, 100)))

### Need to Modify pre-processing function, otherwise self._observation_space produces mismatch produces broadcast error during learning

In [6]:
model = DQN('MlpPolicy', venv)
model.learn(total_timesteps=10000,
            progress_bar=True)

ValueError: could not broadcast input array from shape (8,) into shape (3,4)